In [1]:
import os
import urllib.request
import zipfile
from pyspark import SparkContext
from pyspark.streaming import StreamingContext
import json
import time
from datetime import datetime

In [2]:
from elasticsearch import Elasticsearch

In [3]:
es = Elasticsearch('elasticsearch-node:9200')

if es.indices.exists('stream-test'):
    es.indices.delete('stream-test')
    
    body={
        'mappings': {
            'properties': {
                'count': {'type': 'text'},
                'name': {'type': 'text'},
                'value': {'type': 'text'},
                'timestamp': {'type': 'text'}
            }
        }
    }
    
    es.indices.create(index='stream-test', body=body)
    
print(es.info())

{'name': '927d8d22d8c7', 'cluster_name': 'docker-cluster', 'cluster_uuid': '038uq0CNScCWYvn2HwLEoA', 'version': {'number': '7.5.2', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '8bec50e1e0ad29dad5653712cf3bb580cd1afcdf', 'build_date': '2020-01-15T12:11:52.313576Z', 'build_snapshot': False, 'lucene_version': '8.3.0', 'minimum_wire_compatibility_version': '6.8.0', 'minimum_index_compatibility_version': '6.0.0-beta1'}, 'tagline': 'You Know, for Search'}


In [4]:
os.environ['PYSPARK_SUBMIT_ARGS'] = '--jars elasticsearch-hadoop-7.5.2/dist/elasticsearch-spark-20_2.11-7.5.2.jar pyspark-shell'
sc = SparkContext(appName="PythonSparkStreaming")
sc.setLogLevel("WARN")

In [5]:
#ssc = StreamingContext(sc, 3)

In [6]:
print(os.getcwd())

json_folder_path = ('./sample/')
json_files = [ x for x in os.listdir(json_folder_path ) if x.endswith("json") ]

for json_file in json_files:
    json_file_path = os.path.join(json_folder_path, json_file)
    with open (json_file_path) as f:
        print(json_file_path)
        data_dict = json.loads(f.read())
        print(data_dict)
        
#stream = ssc.textFileStream(json_file_path)        

/home/jovyan
./sample/stream-sample0.22323477920407253.json
{'count': 5, 'name': 'Bilbo', 'value': 37, 'timestamp': 1581297174}
./sample/stream-sample0.0529028610560347.json
{'count': 3, 'name': 'Bilbo', 'value': 66, 'timestamp': 1581297172}
./sample/stream-sample0.5022360747241578.json
{'count': 4, 'name': 'Legolas', 'value': 52, 'timestamp': 1581297173}
./sample/stream-sample0.15689285269925923.json
{'count': 11, 'name': 'Frodo', 'value': 4, 'timestamp': 1581297180}
./sample/stream-sample0.8874662183716416.json
{'count': 7, 'name': 'Legolas', 'value': 56, 'timestamp': 1581297176}
./sample/stream-sample0.8519626057621469.json
{'count': 8, 'name': 'Aragorn', 'value': 78, 'timestamp': 1581297177}
./sample/stream-sample0.5509236364739196.json
{'count': 0, 'name': 'Legolas', 'value': 54, 'timestamp': 1581297169}
./sample/stream-sample0.47629301084795383.json
{'count': 6, 'name': 'Samwise', 'value': 54, 'timestamp': 1581297175}
./sample/stream-sample0.3436550314136364.json
{'count': 9, 'na

In [7]:
es_write_conf = {
    "es.nodes" : '927d8d22d8c7',
    "es.port" : '9200',
    "es.resource" : 'stream-test',
    "es.input.json" : "yes",
    "es.mapping.id": "count"
}

In [8]:
rdd = sc.parallelize(data_dict)

In [9]:
def format_data(x):
    return (data_dict['count'], json.dumps(data_dict))

rdd = rdd.map(lambda x: format_data(x))

In [10]:
rdd.saveAsNewAPIHadoopFile(
    path='-',
    outputFormatClass="org.elasticsearch.hadoop.mr.EsOutputFormat",
    keyClass="org.apache.hadoop.io.NullWritable",
    valueClass="org.elasticsearch.hadoop.mr.LinkedMapWritable",
    conf=es_write_conf)

In [11]:
es_read_conf = {
    "es.nodes" : '927d8d22d8c7',
    "es.port" : '9200',
    "es.resource" : 'stream-test'
    }

In [12]:
es_rdd = sc.newAPIHadoopRDD(
    inputFormatClass="org.elasticsearch.hadoop.mr.EsInputFormat",
    keyClass="org.apache.hadoop.io.NullWritable",
    valueClass="org.elasticsearch.hadoop.mr.LinkedMapWritable",
    conf=es_read_conf)

In [13]:
es_rdd.take(5)

[('12',
  {'count': '12',
   'name': 'Legolas',
   'value': '57',
   'timestamp': '1581297181'})]